In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from math import sqrt, log2, ceil

In [ ]:
### CCS 2023 (Times on PC 2023-01-18 and 2023-01-20)
# Sizes by simulation in sqlite (with main_params_test.py)

# 10K ... 10M
N_arr = [10000, 100000, 1000000, 10000000]
g1_size = 49
g2_size = 97
gT_size = 384

ct_bytes_basic = [g1_size+g2_size+2*gT_size]*len(N_arr)
basic = {
    # Time (s)
    "Setup": [0.0831, 0.215, 0.722, 1.961],
    "Gen": [0.00966, 0.0311, 0.0965, 0.308],
    "Reg": [0.0865, 0.245, 0.732, 2.297],
    # Time (ms)
    "Enc": [2.169, 2.191, 2.167, 2.193],
    "Upd": [1.914, 5.825, 17.722, 56.844],
    "Dec": [3.158, 3.209, 3.164, 3.195],
    # Size (kB)
    "|crs|": [40.960, 114.688, 335.872, 1044.480],
    "|pp|": [16.384, 28.672, 65.536, 188.416],
    "|aux|": [57819, 1858318, 59310518, 1883173015.552],
    # Size (kB)
    "|ct|": [b/1000 for b in ct_bytes_basic],
}
efficient = {
    # Time (s)
    "Setup": [0.156, 0.366, 1.521, 10.880],
    "Gen": [0.00999, 0.0314, 0.0988, 0.319],
    "Reg": [0.0940, 0.256, 0.758, 2.396],
    # Time (ms)
    "Enc": [12.586, 16.001, 17.865, 21.399],
    "Upd": [0.461, 0.555, 0.636, 0.766],
    "Dec": [130.404, 218.789, 281.478, 401.872],
    # Size (kB)
    "|crs|": [40.960, 114.688, 335.872, 1044.480],
    "|pp|" : [106, 258, 716, 2585],
    "|aux|": [aux*2 for aux in [4588, 59818, 666423, 8168354]],
    # Size (kB)
    "|ct|": [ct*sqrt for ct,sqrt in zip(basic["|ct|"],[log2(sqrt(N)) for N in N_arr])],
}
ci_values = {
    "base": {
        # (s)
        "Setup": [0,0,0,0],
        "Gen": [0.0000354, 0.0000852, 0.000118, 0.000186],
        "Reg": [0.000292, 0.000736, 0.000733, 0.0006879],
        # (ms)
        "Enc": [0.00994, 0.00713, 0.00301, 0.00158],
        "Upd": [0.122, 0.219, 0.388, 0.714],
        "Dec": [0.122, 0.0105, 0.00217, 0.00109],
    },
    "efficient": { # TODO update
        # (s)
        "Setup": [0,0,0,0],
        "Gen": [0,0,0,0],
        "Reg": [0,0,0,0],
        # (ms)
        "Enc": [0,0,0,0],
        "Upd": [0,0,0,0],
        "Dec": [0,0,0,0],
    }
}
bigos = {
    "O(\sqrt{N})": (lambda x : sqrt(x)),
}

In [ ]:
# # FOR REFERENCE: Calculated parameter sizes
# basic["|crs|"] = [(2*ceil(sqrt(N))-1)*(g1_size+g2_size)/1000 for N in N_arr]
# basic["|pp|"] = [(g1_size*ceil(N/sqrt(N)))/1000 for N in N_arr]
# basic["|aux|"] = [g1_size*N*sqrt(N)/1000 for N in N_arr]

# efficient["|crs|"] = basic["|crs|"]
# efficient["|pp|"] = [log2(sqrt(N))*ceil(N/sqrt(N))*g1_size/1000 for N in N_arr] # worst case
# efficient["|aux|"] = [N*log2(sqrt(N))*g1_size/1000 for N in N_arr] # worst case

In [ ]:
# # FOR REFERENCE: pp when all N parties have been registered 
# # (note this is not the worst case since N is usually not 2^x-1 for some x)
# import itertools, operator
# def count_bin_ones(x):
#     return list(itertools.accumulate([int(i) for i in bin(x)[2:]], operator.add))[-1]
#
# full_pp = [ceil(N/sqrt(N))*count_bin_ones(ceil(sqrt(N)))*g1_size/1000 for N in N_arr]

In [ ]:
def plot_basic_vs_eff(ax, N_arr, label, ylabels, colors, ci_values={}, cap_width=5):
    
    # save all the legend labels to return
    hs,ls = [],[]
    axs = [ax]
    
    ########################
    # plot regular variant #
    ########################
    ax.plot(N_arr, basic[label][:len(N_arr)], 'o-', label="base", color=colors[0])
    ax.set_xscale("log")
#     if label == "|aux|":
#         # turn off sci notation for y axis
#         current_values = ax.get_yticks()
#         # alternative format string '{:.0f}'
#         ax.set_yticklabels(['{:.0e}'.format(x) for x in current_values])

    # plot CI values if given
    if "base" in ci_values.keys():
        ax.errorbar(N_arr, basic[label][:len(N_arr)], yerr=ci_values["base"][:len(N_arr)], capsize=cap_width)

    ##########################
    # plot efficient variant #
    ##########################
    # set left y-axis label
    if len(ylabels) > 0:
        ax.set_ylabel(ylabels[0])
        
    # plot efficient on different y-axis...
    if len(ylabels) > 1:
        ax2 = ax.twinx()
        ax2.set_ylabel(ylabels[1])
        ax2.plot(N_arr[:len(efficient[label])], efficient[label], 'o-', label="efficient", color=colors[1])
#         if label == "|aux|":
#             # turn off sci notation for y axis
#             current_values = ax2.get_yticks()
#             ax2.set_yticklabels(['{:.0e}'.format(x) for x in current_values])

        # plot CI values if given
        if "efficient" in ci_values.keys():
            ax2.errorbar(N_arr, efficient[label][:len(N_arr)], yerr=ci_values["efficient"][:len(N_arr)], color=colors[1], ecolor=colors[1], capsize=cap_width)

        # axes colors to match series
        ax.spines['left'].set_color(colors[0])
        ax2.spines['left'].set_color(colors[0])
        ax.yaxis.label.set_color(colors[0])
        ax.tick_params(axis='y', colors=colors[0])
        ax.spines['right'].set_color(colors[1])
        ax2.spines['right'].set_color(colors[1])
        ax2.yaxis.label.set_color(colors[1])
        ax2.tick_params(axis='y', colors=colors[1])
        axs.append(ax2)
    # ...or plot it on the same y-axis as the base variant
    else:
        ax.plot(N_arr[:len(efficient[label])], efficient[label], 'o-', label="efficient", color=colors[1])
        # plot CI values if given
        if "efficient" in ci_values.keys():
            ax.errorbar(N_arr, efficient[label][:len(N_arr)], yerr=ci_values["efficient"][:len(N_arr)], capsize=cap_width)
    
    # return axes and handles
    for ax in axs:
        for h,l in zip(*ax.get_legend_handles_labels()):
            hs.append(h)
            ls.append(l)

    ax.set_title(label)
    return hs, ls

In [ ]:
# PLOT TIMES
time_colors = ["tab:blue", "tab:orange"]

fig = plt.figure()

row1 = fig.add_subplot(211)
row1.set_ylabel("Time (s)")
# Turn off axis lines and ticks of row1
row1.spines['top'].set_color('none')
row1.spines['left'].set_color('none')
row1.spines['right'].set_color('none')
row1.spines['bottom'].set_color('none')
row1.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

row2 = fig.add_subplot(212)
row2.set_ylabel("Time (ms)")
row2.set_xlabel("N (log scale)")
# Turn off axis lines and ticks of row2
row2.spines['top'].set_color('none')
row2.spines['left'].set_color('none')
row2.spines['right'].set_color('none')
row2.spines['bottom'].set_color('none')
row2.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

plot_order = ["Setup", "Gen", "Reg", "Enc", "Upd", "Dec"]
for i in range(len(plot_order)):
    sub = fig.add_subplot(2,3,i+1)
    ylabels = []
    if plot_order[i] == "Upd":
        # plot base and eff on different y-axes
        ylabels = ["", ""]
        
#     ci_for_plot = {key: ci_values[key][plot_order[i]] for key in list(ci_values.keys())}
    ci_for_plot = {}
    handles, labels = plot_basic_vs_eff(sub, N_arr, plot_order[i], ylabels, time_colors, ci_for_plot)
    
    # only show legend in the first subgraph
    if i == 0:
        sub.legend(handles, labels)

fig.set_figheight(7)
fig.set_figwidth(15)
plt.tight_layout()
plt.savefig("graph_times.png")

In [ ]:
# PLOT SIZES
size_colors = ["tab:blue", "tab:orange"]

fig = plt.figure()

big = fig.add_subplot(111)
big.set_ylabel("Size (kB)")
big.set_xlabel("N (log scale)")
# Turn off axis lines and ticks of the big subplot
big.spines['top'].set_color('none')
big.spines['left'].set_color('none')
big.spines['right'].set_color('none')
big.spines['bottom'].set_color('none')
big.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)

plot_order = ["|crs|", "|pp|", "|aux|", "|ct|"]
for i in range(len(plot_order)):
    sub = fig.add_subplot(1,4,i+1)
    ylabels = []
    if plot_order[i] == "|aux|":
        # plot base and eff on different y-axes
        ylabels = ["", ""]
        
    handles, labels = plot_basic_vs_eff(sub, N_arr, plot_order[i], ylabels, size_colors)
        
fig.set_figheight(3)
fig.set_figwidth(15)
plt.tight_layout()
plt.savefig("graph_sizes.png")

In [ ]:
# REFERENCE: BIG-Os
bigos_basic = {
    "Setup": (lambda x : sqrt(x)),
    "Gen": (lambda x : sqrt(x)),
    "Reg": (lambda x : sqrt(x)),
    "Enc": (lambda x : 1),
    "Upd": (lambda x : 1),
    "Dec": (lambda x : 1),
    "|crs|": (lambda x : sqrt(x)),
    "|pp|": (lambda x : sqrt(x)),
    "|aux|": (lambda x : x*sqrt(x)),
    "|pk|": (lambda x : 1),
    "|sk|": (lambda x : 1),
    "|ct|": (lambda x : 1),
}
bigos_eff = {
    "Setup": (lambda x : sqrt(x)),
    "Gen": (lambda x : sqrt(x)),
    "Reg": (lambda x : sqrt(x)*log2(sqrt(x))),
    "Enc": (lambda x : log2(sqrt(x))),
    "Upd": (lambda x : log2(sqrt(x))),
    "Dec": (lambda x : log2(sqrt(x))),
    "|crs|": (lambda x : sqrt(x)),
    "|pp|": (lambda x : sqrt(x)*log2(sqrt(x))),
    "|aux|": (lambda x : x*log2(sqrt(x))),
    "|pk|": (lambda x : 1),
    "|sk|": (lambda x : 1),
    "|ct|": (lambda x : log2(sqrt(x))),
}

def plot_asymptotics(N_arr, plot_order, filename=""):
    colors = ["tab:blue", "tab:orange"]

    fig = plt.figure()

    big = fig.add_subplot(111)
    big.set_xlabel("N (log scale)")
    
    # Turn off axis lines and ticks of the big subplot
    big.spines['top'].set_color('none')
    big.spines['left'].set_color('none')
    big.spines['right'].set_color('none')
    big.spines['bottom'].set_color('none')
    big.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    
    # plot asyms
    for i in range(len(plot_order)):
        if len(plot_order) == 6:
            rows = 2
            cols = 3
        else:
            rows = 1
            cols = len(plot_order)
            
        ax = fig.add_subplot(rows,cols,i+1)
        ax.set_title(plot_order[i])
        ax.set_xscale("log")
        
        xs = np.linspace(N_arr[0],N_arr[-1],1000)
    
        ### basic
        ys = [bigos_basic[plot_order[i]](x) for x in xs]
        ax.plot(xs,ys, '--', label="Basic", color=colors[0])
        
        ### efficient
        ys = [bigos_eff[plot_order[i]](x) for x in xs]
        
        # plot aux on two axes
        if plot_order[i] == "|aux|":
            ax2 = ax.twinx()
            # axes colors to match series
            ax.spines['left'].set_color(colors[0])
            ax2.spines['left'].set_color(colors[0])
            ax.yaxis.label.set_color(colors[0])
            ax.tick_params(axis='y', colors=colors[0])
            ax.spines['right'].set_color(colors[1])
            ax2.spines['right'].set_color(colors[1])
            ax2.yaxis.label.set_color(colors[1])
            ax2.tick_params(axis='y', colors=colors[1])
            
            ax2.plot(xs,ys,'--',label="Efficient",color=colors[1])
        else:
            ax.plot(xs,ys, '--', label="Efficient", color=colors[1])
        
    fig.set_figheight(3.5*rows)
    fig.set_figwidth(15)
    plt.tight_layout()
    if filename != "":
        plt.savefig("{}.png".format(filename))

In [ ]:
time_plots = ["Setup", "Gen", "Reg", "Enc", "Upd", "Dec"]
plot_asymptotics(N_arr, time_plots, "asymptotics_time")

In [ ]:
size_plots = ["|crs|", "|pp|", "|aux|", "|ct|"]
plot_asymptotics(N_arr, size_plots, "asymptotics_sizes")